# Part 1: Cross validation

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

In [3]:
import warnings
warnings.filterwarnings('ignore')

In this part we follow the first part of the lab of Section 4 of our textbook to learn how to implement cross validation in Python (see [here](https://islp.readthedocs.io/en/latest/labs/Ch05-resample-lab.html) for the original version of the Lab)


## 1a) Validation set approach

**Objective of part 1a)**: Use the validation set approach to evaluate the performance of a model predicting `mpg` in the `Auto` dataset based on predictor `horsepower`.

In [3]:
# Run this cell to load the dataset
Auto =  load_data('Auto')

### Task 1.1: 
Split the dataset into a train and a test part. Your test set should have 196 samples.

In [4]:
train, validation = train_test_split(Auto, test_size=196)

In [5]:
train

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
109,22.0,4,108.0,94,2379,16.5,73,3,datsun 610
390,28.0,4,120.0,79,2625,18.6,82,1,ford ranger
218,33.5,4,85.0,70,1945,16.8,77,3,datsun f-10 hatchback
7,14.0,8,440.0,215,4312,8.5,70,1,plymouth fury iii
387,27.0,4,140.0,86,2790,15.6,82,1,ford mustang gl
...,...,...,...,...,...,...,...,...,...
147,24.0,4,120.0,97,2489,15.0,74,3,honda civic
168,23.0,4,140.0,78,2592,18.5,75,1,pontiac astro
349,33.0,4,105.0,74,2190,14.2,81,2,volkswagen jetta
352,32.9,4,119.0,100,2615,14.8,81,3,datsun 200sx


In [6]:
validation

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
359,26.6,8,350.0,105,3725,19.0,81,1,oldsmobile cutlass ls
351,32.4,4,108.0,75,2350,16.8,81,3,toyota corolla
21,24.0,4,107.0,90,2430,14.5,70,2,audi 100 ls
344,37.7,4,89.0,62,2050,17.3,81,3,toyota tercel
12,15.0,8,400.0,150,3761,9.5,70,1,chevrolet monte carlo
...,...,...,...,...,...,...,...,...,...
228,15.5,8,350.0,170,4165,11.4,77,1,chevrolet monte carlo landau
322,40.8,4,85.0,65,2110,19.2,80,3,datsun 210
293,35.7,4,98.0,80,1915,14.4,79,1,dodge colt hatchback custom
280,22.3,4,140.0,88,2890,17.3,79,1,ford fairmont 4


### Task 1.2:
Train a simple linear regression model on the training set which predicts `mpg` based on the unique predictor `horsepower`. Make sure to use the training set to train your model.

In [7]:
design = MS(['horsepower']).fit(train)
X_train = design.transform(train)
X_train

,intercept,horsepower
109,1.0,94
390,1.0,79
218,1.0,70
7,1.0,215
387,1.0,86
...,...,...
147,1.0,97
168,1.0,78
349,1.0,74
352,1.0,100


In [8]:
y_train = train['mpg']

In [9]:
model = sm.OLS(y_train,X_train)
results = model.fit()
summarize(results)

,coef,std err,t,P>|t|
intercept,38.7652,1.005,38.567,0.0
horsepower,-0.1470,0.009,-16.256,0.0


### Task 1.3: 
Now use the validation set to compute an estimate for the model's MSE (mean squared error).

In [11]:
y_valid = validation.mpg
X_valid = design.transform(validation)
y_valid_predicted = results.predict(X_valid)
MSE = np.mean((y_valid - y_valid_predicted)**2)
MSE

23.753178493666372

### Task 1.4:
Below you find a preimplemented function `evalMSE()`. Explain what this function does (what are the arguments, what is the output)?

In [12]:
def evalMSE(predictors,
           train,
           validation):
    # build design matrix and response vector
    design = MS(predictors).fit(train)
    X_train = design.transform(train)
    y_train = train.mpg 

    # train model
    model = sm.OLS(y_train,X_train)
    results = model.fit()

    # compute MSE
    y_valid_actual = validation.mpg
    X_validation = design.transform(validation)
    y_valid_predicted = results.predict(X_validation)
    MSE = np.mean((y_valid_actual - y_valid_predicted)**2)
    return MSE

In [13]:
evalMSE(['horsepower'], train, validation)

23.753178493666372

*Your explanation here*

### Task 1.5:

Use the function `evalMSE()` to estimate the MSE on the validation set for linear regression models including successively higher polynomial terms of `horsepower` ranging from degree 1 to degree 3.

In [14]:
from ISLP.models import poly
MSE = []
for i in range(1,4):
    predictors = [poly('horsepower',degree=i)]
    err = evalMSE(predictors, train, validation)
    MSE.append(err)

In [15]:
MSE

[23.753178493666372, 19.039544233087433, 18.998421414459045]

## 1b) Cross validation

Cross validation is implemented most comfortably in `scikit-learn`. In order to use the `scikit-learn` implementation of cross validation with our `statsmodels` linear model, we use the wrapper `sklearn_sm` provided by the `ISLP` library. From the lab in Chapter 5:

"The class `sklearn_sm()` has as its first argument a model from `statsmodels`. It can take two additional optional arguments: `model_str` which can be used to specify a formula, and `model_args` which should be a dictionary of additional arguments used when fitting the model. For example, to fit a logistic regression model we have to specify a family argument. This is passed as `model_args={'family':sm.families.Binomial()}`."

After specifying our design matrix `X` and the vector `y` we call the `scikit-learn` function `cross_validate` (see [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html)).

The result is a dictionary which among others contains the `test_score` which we are interested when we use cross validation to estimate the test error.

### Task 1.6:
Run the following cell to initialize an `sklearn_sm` object with a `statsmodels.OLS` model and a `ModelSpec` object with specified predictor `horsepower`, with dataset `X` with removed `mpg` column und with response vector `y`.

In [16]:
model = sklearn_sm(sm.OLS,
                  MS(['horsepower']))
X = Auto.drop(columns = ['mpg'])
y = Auto['mpg']

The function `cross_validate` is the original `scikit-learn` function which carries out cross validation. We provide the following arguments:
- a model which needs `fit()` and `predict()` methods
- a design matrix `X` and a vector of training labels `y`
- the parameter `cv` specifying the number of folds for cross validation.

See the [official documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html) for more details.

### Task 1.7:
Run a Leave-one-out-cross-validation for a simple linear regression model predicting `mpg` based on `horsepower`. To do so, choose as many folds as your dataset has samples using the parameter `cv`.

In [17]:
cv_results = cross_validate(model, X, y, cv=X.shape[0])

In [18]:
cv_err = np.mean(cv_results['test_score'])
cv_err

24.23151351792922

### Task 1.8:

We can repeat this procedure to compare different models. In the following we do this with various polynomial fits.

Compute the LOOCV error for models to predict `mpg` based on polynomial powers of `mpg` for degrees 1 to 5.

In [21]:
cv_error = np.zeros(5) #initialize empty vector for CV errors of 5 models
M = sklearn_sm(sm.OLS)
y = Auto['mpg']

# loop over 5 different polynomial models
for i in range(1,6):
    # choose power of horsepowers as predictors up to degree i+1
    predictors = [poly('horsepower', degree=i)]
    X = MS(predictors).fit_transform(Auto)
    #perform cross validation for polynomial model of degree i+i
    cv_results = cross_validate(M,
                               X,
                               y,
                               cv = Auto.shape[0])
    # save cross validation error into result vector
    cv_error[i-1] = np.mean(cv_results['test_score'])
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443031, 19.03321385])

### Task 1.9:

Instead of using $K = n$ folds such as above (resulting in Leave-One-Out-Cross-Validation (LOOCV)) we can also specify a smaller integer $K$ of folds. There are two possibilities for this:
- set `cv = K`,
- specify a cross validation generator such as `KFold` (see [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html))

It is recommended to use the second approach, which is generally preferred as we do better control the kind of split when using `KFold`.

Compute the cross validation error for models to predict `mpg` based on polynomial powers of `mpg` for degrees 1 to 5. Use 10 folds.

In [23]:
cv_error = np.zeros(5)

cross_val = KFold(n_splits=10,
                 shuffle=True,
                 random_state=42)

M = sklearn_sm(sm.OLS)
y = Auto['mpg']

# loop over 5 different polynomial models
for i in range(1,6):
    # choose power of horsepowers as predictors up to degree i+1
    predictors = [poly('horsepower', degree=i)]
    X = MS(predictors).fit_transform(Auto)
    #perform cross validation for polynomial model of degree i+i
    cv_results = cross_validate(M,
                               X,
                               y,
                               cv = cross_val)
    # save cross validation error into result vector
    cv_error[i-1] = np.mean(cv_results['test_score'])
cv_error

array([24.1998082 , 19.22863661, 19.26626535, 19.35109227, 19.02323325])

# Part 2: Case study cross validation

(see Exercise 5.4.5)

In this case study we use the credit card dataset to predict the probability of default. We will build a logistic regression model and estimate its test error using the validation set approach and the cross-validation approach.

In [ ]:
# run this cell to load the data
Default = load_data('Default')
Default

Background information on the dataset can be found [in the documentation](https://islp.readthedocs.io/en/latest/datasets/Default.html).

## Task 2.1
Fit a logistic regression model that uses `income` and `balance` to predict `default`.

In [ ]:
# your code here
...

## Task 2.2
Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:

i. Split the sample set into a training set and a validation set.

ii. Fit a multiple logistic regression model using only the training observations.

iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.

iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.

In [ ]:
# your code here
...

## Task 2.3
Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.

In [ ]:
# your code here
...

## Task 2.4
Now predict the test error of the model using 10-fold cross-validation. To do so, follow the steps we developed in Part 1b) of this notebook.

**Note**: To carry out this task, we need to define our own custom score using [`sklearn.make_scorer()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html) and pass this to the function [`sklearn.cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html) by specifying it as an argument to the `scoring` parameter.

Our custom scoring function needs to have the signature `score_func(y, y_pred, **kwargs)` with `y` being the true labels and `y_pred` the predicted labels as output by `sm.Logit()` when applying the `predict()` method. Since `statsmodels` outputs probabilities rather than actual labels, we first transform these probabilities  into labels. This is what our scoring function `accuracy_score_sm()` does.

In [ ]:
# Step 1: Define custom scorer which computes accuracy based on probabilities as
# output by statsmodels
from sklearn.metrics import make_scorer
def accuracy_score_sm(y_true, y_pred_prob):
    # computes the accuracy for the output of a binary classification model

    # inputs:
    #    y_true: ground truth labels, encoded as 0,1
    #    y_pred_prob: probabilities for label 1 as predicted by the model

    # output:
    #    percentage of models 
    y_pred = ...
    return ...
accuracy_sm = make_scorer(...)

# Step 2: Initialize splitter for cross validation and model
cross_val = ...
M = sklearn_sm(sm.Logit)

# Step 3: Define response variable and design matrix for cross validation
y = ...
predictors = ...
X = ...

# Step 4: Run cross validation
cv_results = cross_validate(M,
                            X,
                            y,
                            scoring = accuracy_sm,
                            cv = cross_val)
print('Mean accuracy: ', np.mean(cv_results['test_score']))

## Task 2.5
Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using 10-fold cross-validation. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.

In [ ]:
# your code here
...

# Part 3: Implementation of bootstrap

## Bootstrapping for estimating the accuracy of a statistic
### Introducing the dataset
We closely follow an example presented in [Computational and Inferential Thinking](https://inferentialthinking.com/chapters/13/3/Confidence_Intervals.html).

In [4]:
url = 'https://drive.google.com/uc?id='
file_id = "15xUDQPqkzKJBoxrafC9iNz4EgFlwbmM_"
births = pd.read_csv(url + file_id)
births

,Birth Weight,Gestational Days,Maternal Age,Maternal Height,Maternal Pregnancy Weight,Maternal Smoker
0,120,284,27,62,100,False
1,113,282,33,64,135,False
2,128,279,28,64,115,True
3,108,282,23,67,125,True
4,136,286,25,62,93,False
...,...,...,...,...,...,...
1169,113,275,27,60,100,False
1170,128,265,24,67,120,False
1171,130,291,30,65,150,True
1172,125,281,21,65,110,False


#### Task 3.1

Create a new column `Birth Weight (g)` which contains the birth weight in g. Use the fact 1oz = 28.3495g for your computations.

In [ ]:
#your code here
...

Birth weight is an important factor in the health of a newborn infant. Smaller babies tend to need more medical care in their first days than larger newborns. It is therefore helpful to have an estimate of birth weight before the baby is born. One way to do this is to examine the relationship between birth weight and the number of gestational days.

A simple measure of this relationship is the ratio of birth weight to the number of gestational days. The table ratios contains the first two columns of baby, as well as a column of the ratios. The first entry in that column was calculated as follows:
$$ \frac{3401.94 \text{oz}}{284 \text{ days}} \approx 11.98 \text{g} \text{ per day}.$$

In [ ]:
ratios = pd.DataFrame({
    'Birth Weight' : births['Birth Weight (g)'],
    'Ratio BW:GD' : births['Birth Weight (g)'] / births['Gestational Days']
})
ratios

#### Task 3.2

Plot a histogram of the ratios.

In [ ]:
#your code here

#### Task 3.3

Compute the median ratio and the maximum ratio in the sample.

In [ ]:
#your code here

### Estimating the variability of the population median
We now want to estimate the population median. For this we are going to use the bootstrapping method.
We start by reviewing the idea in a graphical manner:
![bootstrap.png](bootstrap.png)

#### Task 3.4

Define a function `one_bootstrap_median` which will bootstrap the sample and return the median ratio in the bootstrapped sample.

- To bootstrap the sample use the Pandas.DataFrame method [`sample`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html). *Important*: Make sure to draw a sample of the same length as our original sample and make sure to sample with replacement.
- To compute the appropriate quantile, use the Numpy method [`quantile`](https://numpy.org/doc/stable/reference/generated/numpy.quantile.html)

In [ ]:
def one_bootstrap_median():
    ...

#### Task 3.5

Initialize a Numpy vector `bootstrap_medians` with zeros of length 5000 (use the Numpy method [`zeros()`](https://numpy.org/doc/stable/reference/generated/numpy.zeros.html#numpy.zeros). Then fill this vector with 5000 bootstrapped medians.

In [ ]:
bootstrap_medians = ...
...

In [ ]:
fig,ax  = plt.subplots(figsize =(8,8))
sns.histplot(x = bootstrap_medians, bins=25);

## Bootstrapping for estimating the accuracy of a Linear Regression Model
Now, we discuss how to use bootstrapping in order to assess the variability of the coefficient estimates and predictions from a statistical learning method. As an example, we look at a simple linear regression model based on the `Auto` dataset which predicts the `mpg` variable based on `horsepower`.

With the bootstrap method we are going to estimate the distribution of the coefficient for `mpg` in this model and we compare the standard error of this coefficient as estimated by `statsmodels` with our bootstrap estimate.

In [ ]:
Auto = load_data('Auto')

#### Task 3.6 

Define a function `one_bootstrap_model_coefficient` which creates a single bootstrap sample from the Auto dataframe, computes a regression model based on the single predictor `horsepower` and returns the model coefficient for `horsepower`.

In [ ]:
def one_bootstrap_model_coefficient():
    ...

#### Task 3.7

Initialize a Numpy vector `bootstrap_model_coefficients` with zeros of length 5000 (use the Numpy method [`zeros()`](https://numpy.org/doc/stable/reference/generated/numpy.zeros.html#numpy.zeros). Then fill this vector with 5000 bootstrapped model coefficients.

In [ ]:
bootstrap_model_coefficients = ...
...

In [ ]:
fig,ax  = plt.subplots(figsize =(8,8))
sns.histplot(x = bootstrap_model_coefficients, bins=25);

#### Task 3.8 

Estimate the standard error of the model coefficient for `horsepower` and assign it to the variable `standard_error_bootstrap`. Compare to the `sm.OLS()` estimate which should be assigned to the variable `standard_error_bootstrap`.

In [ ]:
standard_error_bootstrap = ...

# compute statsmodels OLS model using the full dataset
...
standard_error_statsmodels = ...

print('Bootstrapped standard error for model coefficient:', "{:10.4f}".format(standard_error_bootstrap))
print('Statsmodels OLS standard error estimate for model coefficient:', "{:10.4f}".format(standard_error_statsmodels))

# Part 4: Case study bootstrap

We continue to consider the use of a logistic regression model to predict the probability of default using income and balance on the `Default` data set. In particular, we will now compute estimates for the standard errors of the income and balance logistic regression coefficients in two different ways: 
1. using the bootstrap, and 
2. using the standard formula for computing the standard errors in the sm.GLM() function.

In [ ]:
# run this cell to load the data
Default = load_data('Default')
Default

## Task 4.1
Using the `summarize()` and `sm.GLM()` functions, determine the estimated standard errors for the coefficients associated with `income` and `balance` in a multiple logistic regression model that uses both predictors.

In [ ]:
...

## Task 4.2
Following the bootstrap example in Part 2 above, estimate the standard errors of the logistic regression coefficients for income and balance with the bootstrap.

In [ ]:
# Step 1: Define function to compute one bootstrap sample of the model coefficients
...

In [ ]:
# Step 2: 
...

In [ ]:
# Step 3a: comparison of bootstrap standard errors and standard errors as per statsmodels.Logit() - balance
print('Bootstrap estimation of standard error for balance parameter:   ', 
      '{:6e}'.format(np.std(balance_coefficients))
)
print('Statsmodels estimation of standard error for balance parameter: ', 
      '{:6e}'.format(summarize(results).loc['balance','std err'])
)

In [ ]:
# Step 3b: comparison of bootstrap standard errors and standard errors as per statsmodels.Logit() - income
print('Bootstrap estimation of standard error for income parameter:   ', 
      '{:6e}'.format(np.std(income_coefficients))
)
print('Statsmodels estimation of standard error for income parameter: ', 
      '{:6e}'.format(summarize(results).loc['income','std err'])
)

## Task 4.3

Comment on the estimated standard errors obtained using the `sm.Logit()`/`sm.GLM()` function and using the bootstrap.

*Your comment here*